In [1]:
def h(x):
    return(x[0]*x[1] + x[2]*x[3] + x[4]*x[5] + x[6]*x[7] + x[0]*x[4]*x[8] + l0 + l0)

def lfsr_update_fw(S):
    temp = S[0] + S[7] + S[38] + S[70] + S[81] + S[96] + l0 + l0
    S = S[1:128] + [temp]
    return S

def lfsr_update_bk(S):
    temp = S[127] + S[6] + S[37] + S[69] + S[80] + S[95] + l0 + l0
    S = [temp] + S[0:127]
    return S

def nfsr_update_fw(B,s):
    temp = s + B[0] + B[26] + B[56] + B[91] + B[96] + B[3]*B[67] + B[11]*B[13] + B[17]*B[18]\
    + B[27]*B[59] + B[40]*B[48] + B[61]*B[65] + B[68]*B[84] + B[88]*B[92]*B[93]*B[95] \
    + B[22]*B[24]*B[25] + B[70]*B[78]*B[82] + l0 + l0
    B = B[1:128] + [temp]
    return B

def nfsr_update_bk(B,s):
    temp = s + B[127] + B[25] + B[55] + B[90] + B[95] + B[2]*B[66] + B[10]*B[12] + B[16]*B[17]\
    + B[26]*B[58] + B[39]*B[47] + B[60]*B[64] + B[67]*B[83] + B[87]*B[91]*B[92]*B[94] \
    + B[21]*B[23]*B[24] + B[69]*B[77]*B[81] + l0 + l0
    B = [temp] + B[0:127]
    return B
    
def keystream(S,B):
    z = h([B[12], S[8], S[13], S[20], B[95], S[42], S[60], S[79], S[94]]) + S[93] + B[2] \
    + B[15] + B[36] + B[45] + B[64] + B[73] + B[89] + l0 + l0
    return z
    
def intersection(A,B):
    temp = []
    for i in range(len(A)):
        if A[i] in B:
            temp.append(A[i])
    return temp

In [2]:

state_len = 256
r = [128,128]
V = BooleanPolynomialRing(state_len,['l%d'%i for i in range(r[0])] + ['n%d'%i for i in range(r[1])])
V.inject_variables()
g = list(V.gens())

Defining l0, l1, l2, l3, l4, l5, l6, l7, l8, l9, l10, l11, l12, l13, l14, l15, l16, l17, l18, l19, l20, l21, l22, l23, l24, l25, l26, l27, l28, l29, l30, l31, l32, l33, l34, l35, l36, l37, l38, l39, l40, l41, l42, l43, l44, l45, l46, l47, l48, l49, l50, l51, l52, l53, l54, l55, l56, l57, l58, l59, l60, l61, l62, l63, l64, l65, l66, l67, l68, l69, l70, l71, l72, l73, l74, l75, l76, l77, l78, l79, l80, l81, l82, l83, l84, l85, l86, l87, l88, l89, l90, l91, l92, l93, l94, l95, l96, l97, l98, l99, l100, l101, l102, l103, l104, l105, l106, l107, l108, l109, l110, l111, l112, l113, l114, l115, l116, l117, l118, l119, l120, l121, l122, l123, l124, l125, l126, l127, n0, n1, n2, n3, n4, n5, n6, n7, n8, n9, n10, n11, n12, n13, n14, n15, n16, n17, n18, n19, n20, n21, n22, n23, n24, n25, n26, n27, n28, n29, n30, n31, n32, n33, n34, n35, n36, n37, n38, n39, n40, n41, n42, n43, n44, n45, n46, n47, n48, n49, n50, n51, n52, n53, n54, n55, n56, n57, n58, n59, n60, n61, n62, n63, n64, n65, n66, n67, n68

In [3]:
import time
import csv 
import numpy as np
import gurobipy as gp
from gurobipy import GRB


start2 = time.time()


keys = 35
keys_b = 0
keys_f = keys-keys_b



S = g[:r[0]]
B = g[r[0]:]

R = [114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126 , 127 , 128+3, 128+21, 128+23, 128+24, 128+25, 128+27, 128+28, 128+33, 128+34, 128+35, 128+36, 128+38, 128+39, 128+41, 128+43, 128+56, 128+59, 128+61, 128+62, 128+88, 128+93]
F = [128 + 32, 128 + 37, 128 + 40, 128 + 42, 128 + 44, 128 + 45, 128 + 116, 128 + 117, 128 + 118, 128 + 119, 128 + 121, 128 + 122, 128 + 124, 128 + 126]
G = [i for i in range(state_len) if i not in R]

Rec = [g[i] for i in R]

print(Rec)
for i in F:
    if i < r[0]:
        S[i] = 0
    else:
        B[i-r[0]] = 0
        
        
for i in range(keys_f):
    z = keystream(S,B)
    z_mon = list(z.monomials())
    print("Forward",i)
    for m in z_mon:
        m_var = list(m.variables())
        if len(intersection(m_var,Rec)) > 0:
            print(intersection(m_var,Rec),len(m_var),end = " + ")
    print("\n")
    
    B = nfsr_update_fw(B,S[0])
    S = lfsr_update_fw(S)
    
S = g[:r[0]]
B = g[r[0]:]

for i in F:
    if i < r[0]:
        S[i] = 0
    else:
        B[i-r[0]] = 0

for i in range(keys_b):
    S = lfsr_update_bk(S)
    B = nfsr_update_bk(B,S[0])
    z = keystream(S,B)
    z_mon = list(z.monomials())
    print("Backward -",i+1)
    for m in z_mon:
        m_var = list(m.variables())
        if len(intersection(m_var,Rec)) > 0:
            print(intersection(m_var,Rec),len(m_var),end = " + ")
    print("\n")
    



[l114, l115, l116, l117, l118, l119, l120, l121, l122, l123, l124, l125, l126, l127, n3, n21, n23, n24, n25, n27, n28, n33, n34, n35, n36, n38, n39, n41, n43, n56, n59, n61, n62, n88, n93]
Forward 0
[n36] 1 + 

Forward 1
[n3] 1 + 

Forward 2
[n38] 1 + 

Forward 3
[n39] 1 + 

Forward 4
[n93] 1 + 

Forward 5
[n41] 1 + 

Forward 6
[n21] 1 + 

Forward 7
[n43] 1 + 

Forward 8
[n23] 1 + 

Forward 9
[n21] 2 + [n21] 3 + [n24] 1 + 

Forward 10
[n25] 1 + 

Forward 11
[n23] 2 + [n23] 3 + [n56] 1 + 

Forward 12
[n24] 2 + [n24] 3 + [n27] 1 + 

Forward 13
[n25] 2 + [n25] 3 + [n28] 1 + 

Forward 14
[n59] 1 + 

Forward 15
[n27] 2 + [n27] 3 + [n88] 1 + 

Forward 16
[n28] 2 + [n28] 3 + [n61] 1 + 

Forward 17
[n62] 1 + 

Forward 18
[n33] 1 + 

Forward 19
[n21] 1 + [n34] 1 + 

Forward 20
[n35] 1 + [n56] 1 + [n93] 1 + 

Forward 21
[n33] 2 + [l114] 1 + [n23] 1 + [n36] 1 + 

Forward 22
[n34] 2 + [l115] 1 + [n24] 1 + 

Forward 23
[n35] 2 + [l116] 1 + [n25] 1 + [n38] 1 + [n59] 1 + 

Forward 24
[n36] 2 + [l117]

In [ ]:
128 + 32, 128 + 37, 128 + 40, 128 + 42, 128 + 44, 128 + 45, 128 + 116, 128 + 117, 128 + 118, 128 + 119, 128 + 121, 128 + 122, 128 + 124, 128 + 126